<a href="https://colab.research.google.com/github/moustafa-7/ECG-Project/blob/master/ECG_CNN_using_2D_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

SRC_FLD="mit_arrythmia_dat"
RES_FLD="mit_arrythmia"

# First, download the dataset and save it to mit_arrythmia_dat directory:
mkdir $SRC_FLD
mkdir $RES_FLD

cd $SRC_FLD

wget http://physionet.org/physiobank/database/mitdb/{100..234}.dat
wget http://physionet.org/physiobank/database/mitdb/{100..234}.hea
wget http://physionet.org/physiobank/database/mitdb/{100..234}.atr

cd ..

--2019-08-28 03:55:32--  http://physionet.org/physiobank/database/mitdb/100.dat
Resolving physionet.org (physionet.org)... 128.30.30.88
Connecting to physionet.org (physionet.org)|128.30.30.88|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1950000 (1.9M) [application/octet-stream]
Saving to: ‘100.dat’

     0K .......... .......... .......... .......... ..........  2%  132K 14s
    50K .......... .......... .......... .......... ..........  5%  264K 10s
   100K .......... .......... .......... .......... ..........  7% 44.7M 7s
   150K .......... .......... .......... .......... .......... 10% 9.86M 5s
   200K .......... .......... .......... .......... .......... 13%  271K 5s
   250K .......... .......... .......... .......... .......... 15% 46.6M 4s
   300K .......... .......... .......... .......... .......... 18% 83.9M 3s
   350K .......... .......... .......... .......... .......... 21% 11.2M 3s
   400K .......... .......... .......... .......... ........

In [2]:
!pip install  wfdb
!pip install biosppy

     |████████████████████████████████| 102kB 3.8MB/s 
  Created wheel for wfdb: filename=wfdb-2.2.1-cp36-none-any.whl size=100366 sha256=e3c2728a6ac9e3932b089106c15052f4c9fac5dc016892eebdd41a784b9f9746
  Stored in directory: /root/.cache/pip/wheels/bb/a9/00/0078d26b0c15b31be0001af8eb659496709c361c69641303f1
Successfully built wfdb
     |████████████████████████████████| 81kB 3.0MB/s 
     |████████████████████████████████| 40kB 20.1MB/s 
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl size=5499 sha256=f0b73a948c4dbcffd62a53d9b95b63285234453fa4d3ce623d1b6cccf5816345
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
Successfully built shortuuid


In [0]:
from glob import glob
import wfdb
import numpy as np
from tqdm import tqdm
import biosppy
import matplotlib.pyplot as plt
import cv2
import os
import math
import gc

def get_records():
    """ Get paths for data in data/mit/ directory """
    # Download if doesn't exist

    # There are 3 files for each record
    # *.atr is one of them
    paths = glob('./*.atr')

    # Get rid of the extension
    paths = [path[:-4] for path in paths]
    paths.sort()

    return paths

def segmentation(records, type, output_dir=''):

    """'N' for normal beats. Similarly we can give the input 'L' for left bundle branch block beats. 'R' for right bundle branch block
        beats. 'A' for Atrial premature contraction. 'V' for ventricular premature contraction. '/' for paced beat. 'E' for Ventricular
        escape beat."""
    os.makedirs(output_dir, exist_ok=True)
    results = []
    kernel = np.ones((4, 4), np.uint8)
    count = 1

    '''
    max_values = []
    min_values = []
    mean_values = []
    for e in tqdm(records):
        signals, fields = wfdb.rdsamp(e, channels=[0])
        mean_values.append(np.mean(signals))
    mean_v = np.mean(np.array(mean_values))
    std_v = 0
    count = 0
    for e in tqdm(records):
        signals, fields = wfdb.rdsamp(e, channels=[0])
        count += len(signals)
        for s in signals:
            std_v += (s[0] - mean_v)**2
    std_v = np.sqrt(std_v/count)'''

    mean_v = -0.33859
    std_v = 0.472368
    floor = mean_v - 3*std_v
    ceil = mean_v + 3*std_v

    for e in tqdm(records):
        signals, fields = wfdb.rdsamp(e, channels = [0])

        ann = wfdb.rdann(e, 'atr')
        good = [type]
        ids = np.in1d(ann.symbol, good)
        imp_beats = ann.sample[ids]
        beats = (ann.sample)
        for i in tqdm(imp_beats):
            beats = list(beats)
            j = beats.index(i)
            if(j!=0 and j!=(len(beats)-1)):

                data = (signals[beats[j]-96: beats[j]+96, 0])

                #results.append(data)

                plt.axis([0, 192, floor, ceil])
                plt.plot(data, linewidth=0.5)
                plt.xticks([]), plt.yticks([])
                for spine in plt.gca().spines.values():
                    spine.set_visible(False)

                filename = output_dir + 'fig_{}'.format(count) + '.png'
                plt.savefig(filename)
                plt.close()
                im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                im_gray = cv2.erode(im_gray, kernel, iterations=1)
                im_gray = cv2.resize(im_gray, (192, 128), interpolation=cv2.INTER_LANCZOS4)
                cv2.imwrite(filename, im_gray)
                print('img writtten {}'.format(filename))
                count += 1
                
        
        gc.collect()


    return results

In [0]:
os.chdir('mit_arrythmia_dat')

In [0]:
labels = ['N', 'L', 'R', 'A', 'V', '/', 'E', '!']
output_dirs = ['NOR/', 'LBBB/', 'RBBB/', 'APC/', 'PVC/', 'PAB/', 'VEB', 'VFE']

records = get_records()


for type, output_dir in zip(labels[1:], output_dirs[1:]):
  sgs = segmentation(records, type, output_dir='./MIT-BIH_AD/'+output_dir)






  0%|          | 0/48 [00:00<?, ?it/s]




0it [00:00, ?it/s]








  2%|▏         | 1/48 [00:01<01:06,  1.41s/it]




0it [00:00, ?it/s]









0it [00:00, ?it/s]








  6%|▋         | 3/48 [00:01<00:45,  1.02s/it]




0it [00:00, ?it/s]









0it [00:00, ?it/s]








 10%|█         | 5/48 [00:01<00:31,  1.35it/s]




0it [00:00, ?it/s]








 12%|█▎        | 6/48 [00:01<00:23,  1.82it/s]




0it [00:00, ?it/s]








 15%|█▍        | 7/48 [00:02<00:17,  2.39it/s]




0it [00:00, ?it/s]









0it [00:00, ?it/s]








 19%|█▉        | 9/48 [00:02<00:12,  3.12it/s]




  0%|          | 0/2492 [00:00<?, ?it/s]




  0%|          | 2/2492 [00:00<02:22, 17.45it/s]




  0%|          | 4/2492 [00:00<02:24, 17.19it/s]

img writtten ./MIT-BIH_AD/LBBB/fig_1.png
img writtten ./MIT-BIH_AD/LBBB/fig_2.png
img writtten ./MIT-BIH_AD/LBBB/fig_3.png
img writtten ./MIT-BIH_AD/LBBB/fig_4.png







  0%|          | 6/2492 [00:00<02:28, 16.69it/s]




  0%|          | 8/2492 [00:00<02:28, 16.72it/s]

img writtten ./MIT-BIH_AD/LBBB/fig_5.png
img writtten ./MIT-BIH_AD/LBBB/fig_6.png
img writtten ./MIT-BIH_AD/LBBB/fig_7.png
img writtten ./MIT-BIH_AD/LBBB/fig_8.png







  0%|          | 10/2492 [00:00<02:30, 16.45it/s]




  0%|          | 12/2492 [00:00<02:30, 16.49it/s]

...

img writtten ./MIT-BIH_AD/RBBB/fig_3685.png
img writtten ./MIT-BIH_AD/RBBB/fig_3686.png
img writtten ./MIT-BIH_AD/RBBB/fig_3687.png
img writtten ./MIT-BIH_AD/RBBB/fig_3688.png







100%|█████████▉| 1524/1531 [01:38<00:00, 15.62it/s]




100%|█████████▉| 1526/1531 [01:38<00:00, 15.78it/s]

img writtten ./MIT-BIH_AD/RBBB/fig_3689.png
img writtten ./MIT-BIH_AD/RBBB/fig_3690.png
img writtten ./MIT-BIH_AD/RBBB/fig_3691.png
img writtten ./MIT-BIH_AD/RBBB/fig_3692.png







100%|█████████▉| 1528/1531 [01:38<00:00, 15.70it/s]




100%|█████████▉| 1530/1531 [01:38<00:00, 15.60it/s]




100%|██████████| 1531/1531 [01:38<00:00, 15.55it/s]

img writtten ./MIT-BIH_AD/RBBB/fig_3693.png
img writtten ./MIT-BIH_AD/RBBB/fig_3694.png
img writtten ./MIT-BIH_AD/RBBB/fig_3695.png


In [0]:
# paper model


from models import proposed_model
from keras.optimizers import Adam
from keras.utils import np_utils

from callbacks import Step
import numpy as np
import random
import cv2
import os
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import glob
import math

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.45
set_session(tf.Session(config=config))


def plot_history(history, result_dir):
    plt.plot(history.history['acc'], marker='.')
    plt.plot(history.history['val_acc'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.close()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.close()
def save_history(history, result_dir):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'result.txt'), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))
        fp.close()

def process_batch(lines,img_path,inputH,inputW,train=True):
    imagew = 128
    imageh = 128
    num = len(lines)
    batch = np.zeros((num, imagew, imageh, 3), dtype='float32')


    labels = np.zeros(num, dtype='int')
    for i in range(num):
        path = lines[i].split(' ')[0]
        label = lines[i].split(' ')[-1]

        label = label.strip('\n')
        label = int(label)

        img = os.path.join(img_path, path)

        if train:
            crop_x = random.randint(0, np.max([0, imagew-inputW]))
            crop_y = random.randint(0, np.max([0, imageh-inputH]))
            is_flip = random.randint(0, 1)


            image = cv2.imread(img)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image[crop_y:crop_y + inputH, crop_x:crop_x + inputW, :]
            image = cv2.resize(image, (imagew, imageh))

            if is_flip == 1:
                image = cv2.flip(image, 1)

            #batch[i][:][:][:] = image[crop_y:crop_y + inputH,crop_x:crop_x + inputW, :]
            batch[i] = image
            labels[i] = label
        else:
            image = cv2.imread(img)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


            Hshmean = int(np.round(np.max([0, np.round((imageh-inputH)/2)])))
            Wshmean = int(np.round(np.max([0, np.round((imagew-inputW)/2)])))
            image = image[Hshmean:Hshmean+inputH,Wshmean:Wshmean+inputW, :]
            image = cv2.resize(image, (imagew, imageh))

            batch[i] = image
            labels[i] = label

    return batch, labels

def generator_train_batch( train_txt, batch_size, num_classes, img_path, inputH, inputW ):
    ff = open(train_txt, 'r')
    lines = ff.readlines()
    num = len(lines)
    while True:
        new_line = []
        index = [n for n in range(num)]
        random.shuffle(index)
        for m in range(num):
            new_line.append(lines[index[m]])

        for i in range(int(num/batch_size)):
            a = i*batch_size
            b = (i+1)*batch_size
            x_train, x_labels = process_batch(new_line[a:b], img_path, inputH, inputW, train=True)
            y = np_utils.to_categorical(np.array(x_labels), num_classes)
            yield x_train, y

def generator_val_batch(val_txt,batch_size,num_classes,img_path,inputH,inputW):
    f = open(val_txt, 'r')
    lines = f.readlines()
    num = len(lines)
    while True:
        new_line = []
        index = [n for n in range(num)]
        random.shuffle(index)
        for m in range(num):
            new_line.append(lines[index[m]])
        for i in range(int(num / batch_size)):
            a = i * batch_size
            b = (i + 1) * batch_size
            y_test,y_labels = process_batch(new_line[a:b],img_path,inputH,inputW,train=False)
            y = np_utils.to_categorical(np.array(y_labels), num_classes)
            yield y_test, y

def generator_train_batch_proposed( new_lines, k, batch_size, num_classes, img_path, inputH, inputW ):

    val_set = 0
    while True:
        if val_set >= k:
            val_set = 0
        else:
            pass

        new_line = []
        for i in range(len(new_lines)):
            if val_set != i:
                new_line += new_lines[i]

        num = len(new_line)
        random.shuffle(new_line)
        for i in range(int(num/batch_size)):
            a = i*batch_size
            b = (i+1)*batch_size
            x_train, x_labels = process_batch(new_line[a:b], img_path, inputH, inputW, train=True)
            y = np_utils.to_categorical(np.array(x_labels), num_classes)
            yield x_train, y
        val_set += 1

def generator_val_batch_proposed(new_lines, k, batch_size, num_classes, img_path, inputH, inputW):


    val_set = 0
    while True:
        if val_set >= k:
            val_set = 0
        else:
            pass

        new_line = new_lines[val_set]
        num = len(new_lines)
        random.shuffle(new_line)

        for i in range(int(num / batch_size)):
            a = i * batch_size
            b = (i + 1) * batch_size
            y_test,y_labels = process_batch(new_line[a:b],img_path,inputH,inputW,train=False)
            y = np_utils.to_categorical(np.array(y_labels), num_classes)
            yield y_test, y
        val_set += 1

def main():
    proposed = True
    if proposed:
        outputdir = 'result/first_attempt/'
        if os.path.isdir(outputdir):
            print('save in :'+outputdir)
        else:
            os.makedirs(outputdir)

        train_img_path = '/home/cc_lee/Dataset/MIT-BIH_AD/'
        train_file = '/home/cc_lee/Documents/ECG-Arrhythmia-classification/MIT-BIH_AD_train.txt'
        num_classes = 8
        k = 10


        f1 = open(train_file, 'r')
        lines = f1.readlines()
        f1.close()

        train_samples = len(lines)
        val_samples = len(lines)//k

        num = len(lines)
        new_lines = []
        index = [n for n in range(num)]
        random.shuffle(index)
        for m in range(num):
            new_lines.append(lines[index[m]])

        lines = new_lines
        temp = []
        new_lines = []
        for i in range(num):
            if i % val_samples == 0:
                temp = []
                new_lines.append(temp)
            temp.append(lines[i])

        batch_size = 1
        epochs = 40
        input_h = 96
        input_w = 96

        model = proposed_model(nb_classes=num_classes)


        lr = 0.0001
        adam = Adam(lr=lr)
        model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
        model.summary()
        history = model.fit_generator(generator_train_batch_proposed(new_lines, k, batch_size, num_classes, train_img_path, input_h, input_w),
                                      steps_per_epoch=train_samples // batch_size,
                                      epochs=epochs,
                                      callbacks=[Step()],
                                      validation_data=generator_val_batch_proposed(new_lines, k, batch_size, num_classes, train_img_path, input_h, input_w),
                                      validation_steps=val_samples // batch_size,
                                      verbose=1)
        plot_history(history, outputdir)
        save_history(history, outputdir)
        model.save_weights(outputdir+'proposed_model_{}.h5'.format(proposed))
    else:
        outputdir = 'result/first_attempt/'
        if os.path.isdir(outputdir):
            print('save in :' + outputdir)
        else:
            os.makedirs(outputdir)

        train_img_path = '/home/cc_lee/Dataset/MIT-BIH_AD/'
        test_img_path = '//home/cc_lee/Dataset/MIT-BIH_AD/'
        train_file = '/home/cc_lee/Documents/ECG-Arrhythmia-classification/MIT-BIH_AD_train.txt'
        test_file = '/home/cc_lee/Documents/ECG-Arrhythmia-classification/MIT-BIH_AD_val.txt'
        num_classes = 8

        f1 = open(train_file, 'r')
        f2 = open(test_file, 'r')
        lines = f1.readlines()
        f1.close()
        train_samples = len(lines)
        lines = f2.readlines()
        f2.close()
        val_samples = len(lines)

        batch_size = 1
        epochs = 40
        input_h = 96
        input_w = 96

        model = proposed_model(nb_classes=num_classes)

        lr = 0.0001
        adam = Adam(lr=lr)
        model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
        model.summary()
        history = model.fit_generator(
            generator_train_batch(train_file, batch_size, num_classes, train_img_path, input_h, input_w),
            steps_per_epoch=train_samples // batch_size,
            epochs=epochs,
            callbacks=[Step()],
            validation_data=generator_val_batch(test_file, batch_size, num_classes, test_img_path, input_h, input_w),
            validation_steps=val_samples // batch_size,
            verbose=1)
        plot_history(history, outputdir)
        save_history(history, outputdir)
        model.save_weights(outputdir+'proposed_model_{}.h5'.format(proposed))

if __name__ == '__main__':
    main()

In [0]:
# Edited model


from models import proposed_model
from keras.optimizers import Adam
from keras.utils import np_utils

from callbacks import Step
import numpy as np
import random
import cv2
import os
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import glob
import math

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.45
set_session(tf.Session(config=config))


def plot_history(history, result_dir):
    plt.plot(history.history['acc'], marker='.')
    plt.plot(history.history['val_acc'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.close()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.close()

def save_history(history, result_dir):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'result.txt'), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))
        fp.close()

def process_batch(lines,img_path,inputH,inputW,train=True):
    imagew = 192
    imageh = 128
    num = len(lines)
    batch = np.zeros((num, inputH, inputW, 3), dtype='float32')


    labels = np.zeros(num, dtype='int')
    for i in range(num):
        path = lines[i].split(' ')[0]
        label = lines[i].split(' ')[-1]

        label = label.strip('\n')
        label = int(label)

        img = os.path.join(img_path, path)

        if train:
            crop_x = random.randint(0, np.max([0, imagew-inputW]))
            image = cv2.imread(img)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image[:, crop_x:crop_x + inputW, :]
            batch[i] = image
            labels[i] = label
        else:
            image = cv2.imread(img)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image[:, 32:32+128, :]
            batch[i] = image
            labels[i] = label

    return batch, labels

def generator_train_batch( train_txt, batch_size, num_classes, img_path, inputH, inputW ):
    ff = open(train_txt, 'r')
    lines = ff.readlines()
    num = len(lines)
    while True:
        new_line = []
        index = [n for n in range(num)]
        random.shuffle(index)
        for m in range(num):
            new_line.append(lines[index[m]])

        for i in range(int(num/batch_size)):
            a = i*batch_size
            b = (i+1)*batch_size
            x_train, x_labels = process_batch(new_line[a:b], img_path, inputH, inputW, train=True)
            y = np_utils.to_categorical(np.array(x_labels), num_classes)
            yield x_train, y

def generator_val_batch(val_txt,batch_size,num_classes,img_path,inputH,inputW):
    f = open(val_txt, 'r')
    lines = f.readlines()
    num = len(lines)
    while True:
        new_line = []
        index = [n for n in range(num)]
        random.shuffle(index)
        for m in range(num):
            new_line.append(lines[index[m]])
        for i in range(int(num / batch_size)):
            a = i * batch_size
            b = (i + 1) * batch_size
            y_test,y_labels = process_batch(new_line[a:b],img_path,inputH,inputW,train=False)
            y = np_utils.to_categorical(np.array(y_labels), num_classes)
            yield y_test, y

def main():

    outputdir = 'result/special_v3/'
    if os.path.isdir(outputdir):
        print('save in :' + outputdir)
    else:
        os.makedirs(outputdir)

    train_img_path = '/data/MIT-BIH_AD_v3/'
    test_img_path = '/data/MIT-BIH_AD_v3/'
    train_file = './MIT-BIH_AD_sp_train.txt'
    test_file = './MIT-BIH_AD_sp_val.txt'
    num_classes = 7

    f1 = open(train_file, 'r')
    f2 = open(test_file, 'r')
    lines = f1.readlines()
    f1.close()
    train_samples = len(lines)
    lines = f2.readlines()
    f2.close()
    val_samples = len(lines)

    batch_size = 32
    epochs = 120
    input_h = 128
    input_w = 128

    model = proposed_model(nb_classes=num_classes)

    lr = 0.0001
    adam = Adam(lr=lr)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.summary()
    history = model.fit_generator(
        generator_train_batch(train_file, batch_size, num_classes, train_img_path, input_h, input_w),
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        callbacks=[Step()],
        validation_data=generator_val_batch(test_file, batch_size, num_classes, test_img_path, input_h, input_w),
        validation_steps=val_samples // batch_size,
        verbose=1)
    plot_history(history, outputdir)
    save_history(history, outputdir)
    model.save_weights(outputdir+'proposed_model')

if __name__ == '__main__':
    main()